In [1]:
import pandas as pd

# Load your cleaned dataset
df = pd.read_csv("PriceLens_clean_dataset.csv")

# Preview the data
print("Shape:", df.shape)
df.head()

Shape: (8003, 9)


,bedrooms,bathrooms,toilets,parking_space,title,town,state,price,listing_type
0,4,5,5,4,Detached Duplex,Lekki,Lagos,120000000,rent
1,4,4,5,6,Detached Duplex,Ajah,Lagos,40000000,rent
2,4,4,5,2,Semi Detached Duplex,Lekki,Lagos,75000000,rent
3,4,5,5,4,Detached Duplex,Lekki,Lagos,65000000,rent
4,4,4,5,5,Detached Duplex,Ajah,Lagos,60000000,rent


In [2]:
# Separate features and target
X = df.drop(columns=['price'])
y = df['price'] 

In [3]:
df.head(15)

,bedrooms,bathrooms,toilets,parking_space,title,town,state,price,listing_type
0,4,5,5,4,Detached Duplex,Lekki,Lagos,120000000,rent
1,4,4,5,6,Detached Duplex,Ajah,Lagos,40000000,rent
2,4,4,5,2,Semi Detached Duplex,Lekki,Lagos,75000000,rent
3,4,5,5,4,Detached Duplex,Lekki,Lagos,65000000,rent
4,4,4,5,5,Detached Duplex,Ajah,Lagos,60000000,rent
5,3,4,5,4,Terraced Duplexes,Lokogoma District,Abuja,25000000,rent
6,4,5,6,3,Semi Detached Duplex,Lekki,Lagos,65000000,rent
7,4,4,5,4,Semi Detached Duplex,Lekki,Lagos,70000000,rent
8,4,4,5,4,Terraced Duplexes,Lekki,Lagos,48000000,rent
9,3,3,4,4,Detached Bungalow,Lokogoma District,Abuja,37000000,rent


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [5]:
categorical_features = X.select_dtypes(include="object").columns.tolist()
numerical_features = X.select_dtypes(exclude="object").columns.tolist()

In [6]:
df.head()

,bedrooms,bathrooms,toilets,parking_space,title,town,state,price,listing_type
0,4,5,5,4,Detached Duplex,Lekki,Lagos,120000000,rent
1,4,4,5,6,Detached Duplex,Ajah,Lagos,40000000,rent
2,4,4,5,2,Semi Detached Duplex,Lekki,Lagos,75000000,rent
3,4,5,5,4,Detached Duplex,Lekki,Lagos,65000000,rent
4,4,4,5,5,Detached Duplex,Ajah,Lagos,60000000,rent


In [7]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pickle


In [8]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import TransformedTargetRegressor
from sklearn.impute import SimpleImputer
import joblib
import numpy as np

# Categorical pipeline
categorical_features = ["title", "town", "state", "listing_type"]
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Full preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'  # For numeric inputs
)

# Final pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', TransformedTargetRegressor(
        regressor=GradientBoostingRegressor(
            n_estimators=200, max_depth=5, learning_rate=0.1, random_state=42),
        func=np.log1p, inverse_func=np.expm1
    ))
])

# Fit your model
model.fit(X_train, y_train)




,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [9]:
y_pred_log = model.predict(X_test)  # Must be from X_test, not y_test

In [10]:
from sklearn.model_selection import train_test_split

# X = your features, y = log-transformed price
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [11]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import TransformedTargetRegressor
from sklearn.impute import SimpleImputer
import joblib
import numpy as np

# Categorical pipeline
categorical_features = ["title", "town", "state", "listing_type"]
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Full preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'  # For numeric inputs
)

# Final pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', TransformedTargetRegressor(
        regressor=GradientBoostingRegressor(
            n_estimators=200, max_depth=5, learning_rate=0.1, random_state=42),
        func=np.log1p, inverse_func=np.expm1
    ))
])

# Set grid parameters
param_grid = {
    'regressor__regressor__n_estimators': [100, 200],
    'regressor__regressor__max_depth': [3, 5],
    'regressor__regressor__learning_rate': [0.05, 0.1],
}

# Fit your model
# Grid search on train set only
grid = GridSearchCV(model, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid.fit(X_train, y_train)
best_model = grid.best_estimator_

In [12]:
y_pred_log = best_model.predict(X_test) 

In [13]:
# Remove sample_weight from features
feature_cols = ['bedrooms', 'bathrooms', 'toilets', 'parking_space', 'title', 'town', 'state', 'listing_type']
X_train_clean = X_train[feature_cols]
X_test_clean = X_test[feature_cols]

# Retrain with clean features
grid_clean = GridSearchCV(model, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_clean.fit(X_train_clean, y_train)
best_model_clean = grid_clean.best_estimator_

# Test again
y_pred_clean = best_model_clean.predict(X_test_clean)
r2_clean = r2_score(y_test, y_pred_clean)
rmse_clean = np.sqrt(mean_squared_error(y_test, y_pred_clean))

print(f"✅ Clean R-squared: {r2_clean:.4f}")
print(f"✅ Clean RMSE: ₦{rmse_clean:,.2f}")

✅ Clean R-squared: 0.7934
✅ Clean RMSE: ₦28,944,213.35


In [14]:
# Save the BEST model (not the original pipeline)
joblib.dump(best_model, 'pricelens_gradient_boosting_model.pkl')
print("🎉 Model saved to 'pricelens_gradient_boosting_model.pkl'")


🎉 Model saved to 'pricelens_gradient_boosting_model.pkl'


In [15]:
# Save feature information for backend reference
feature_info = {
    'features': ['bedrooms', 'bathrooms', 'toilets', 'parking_space', 'title', 'town', 'state', 'listing_type'],
    'categorical_features': ['title', 'town', 'state', 'listing_type'],
    'numerical_features': ['bedrooms', 'bathrooms', 'toilets', 'parking_space']
}
joblib.dump(feature_info, 'model_features.pkl')

['model_features.pkl']

In [16]:
import joblib
import pandas as pd

# Load the dataset to get valid values
df = pd.read_csv("PriceLens_clean_dataset.csv")

# Define the complete model metadata
model_metadata = {
    # Required feature order (CRITICAL - must match exactly)
    'feature_order': ['bedrooms', 'bathrooms', 'toilets', 'parking_space', 'title', 'town', 'state', 'listing_type'],
    
    # Feature types
    'numerical_features': ['bedrooms', 'bathrooms', 'toilets', 'parking_space'],
    'categorical_features': ['title', 'town', 'state', 'listing_type'],
    
    # Valid values for categorical features
    'valid_values': {
        'title': sorted(df['title'].dropna().unique().tolist()),
        'town': sorted(df['town'].dropna().unique().tolist()),
        'state': ['Lagos', 'Abuja'],  # Only these two states
        'listing_type': ['Rent', 'Sale']
    },
    
    # Data types expected
    'expected_dtypes': {
        'bedrooms': 'int',
        'bathrooms': 'int', 
        'toilets': 'int',
        'parking_space': 'int',
        'title': 'object',
        'town': 'object',
        'state': 'object',
        'listing_type': 'object'
    },
    
    # Model performance metrics (for reference)
    'model_performance': {
        'r2_score': 0.7934,
        'rmse': 28944213.35,
        'price_range': {
            'min': 10000000,
            'max': 275000000
        }
    },
    
    # Usage instructions
    'usage_notes': {
        'input_format': 'pandas DataFrame with exact column order',
        'missing_values': 'Not allowed - all fields required',
        'transformations': 'Model handles log transformation automatically',
        'output': 'Price in Nigerian Naira (NGN)'
    }
}

# Save the metadata
joblib.dump(model_metadata, 'model_metadata.pkl')
print("✅ Model metadata saved to 'model_metadata.pkl'")

# Create a sample input template for backend team
sample_input = {
    'bedrooms': 3,
    'bathrooms': 2,
    'toilets': 3,
    'parking_space': 2,
    'title': 'Detached Duplex',
    'town': 'Lekki',
    'state': 'Lagos',
    'listing_type': 'Sale'
}

joblib.dump(sample_input, 'sample_input.pkl')
print("✅ Sample input saved to 'sample_input.pkl'")

# Create backend usage example
backend_example = """
# Backend Usage Example
import joblib
import pandas as pd

# Load model and metadata
model = joblib.load('best_gradient_boosting_model.pkl')
metadata = joblib.load('model_metadata.pkl')

# Create input DataFrame (MUST follow exact order)
input_data = {
    'bedrooms': 3,
    'bathrooms': 2,
    'toilets': 3,
    'parking_space': 2,
    'title': 'Detached Duplex',
    'town': 'Lekki',
    'state': 'Lagos',
    'listing_type': 'Sale'
}

# Convert to DataFrame with correct column order
df = pd.DataFrame([input_data])
df = df[metadata['feature_order']]  # Ensure correct order

# Make prediction
prediction = model.predict(df)[0]
print(f"Predicted Price: NGN {prediction:,.2f}")
"""

# Save usage example with UTF-8 encoding
with open('backend_usage_example.py', 'w', encoding='utf-8') as f:
    f.write(backend_example)

print("✅ Backend usage example saved to 'backend_usage_example.py'")

# Display summary
print("\n🎯 Files created for backend team:")
print("1. best_gradient_boosting_model.pkl - The trained model")
print("2. model_metadata.pkl - Feature order, valid values, performance metrics")
print("3. sample_input.pkl - Example input format")
print("4. backend_usage_example.py - Complete usage example") 

✅ Model metadata saved to 'model_metadata.pkl'
✅ Sample input saved to 'sample_input.pkl'
✅ Backend usage example saved to 'backend_usage_example.py'

🎯 Files created for backend team:
1. best_gradient_boosting_model.pkl - The trained model
2. model_metadata.pkl - Feature order, valid values, performance metrics
3. sample_input.pkl - Example input format
4. backend_usage_example.py - Complete usage example


In [17]:
df.head()

,bedrooms,bathrooms,toilets,parking_space,title,town,state,price,listing_type
0,4,5,5,4,Detached Duplex,Lekki,Lagos,120000000,rent
1,4,4,5,6,Detached Duplex,Ajah,Lagos,40000000,rent
2,4,4,5,2,Semi Detached Duplex,Lekki,Lagos,75000000,rent
3,4,5,5,4,Detached Duplex,Lekki,Lagos,65000000,rent
4,4,4,5,5,Detached Duplex,Ajah,Lagos,60000000,rent


In [18]:
df['title'].unique().tolist()

['Detached Duplex',
 'Semi Detached Duplex',
 'Terraced Duplexes',
 'Detached Bungalow',
 'Block of Flats',
 'Terraced Bungalow',
 'Semi Detached Bungalow']

In [19]:
df['town'].unique().tolist()

['Lekki',
 'Ajah',
 'Lokogoma District',
 'Victoria Island (Vi)',
 'Katampe',
 'Magodo',
 'Kaura',
 'Galadimawa',
 'Gwarinpa',
 'Lugbe District',
 'Ifako-Ijaiye',
 'Agege',
 'Jahi',
 'Orozo',
 'Ikeja',
 'Idu Industrial',
 'Isheri North',
 'Kuje',
 'Isheri',
 'Ikoyi',
 'Life Camp',
 'Ibeju Lekki',
 'Dape',
 'Ejigbo',
 'Guzape District',
 'Ojodu',
 'Gaduwa',
 'Dakwo',
 'Asokoro District',
 'Utako',
 'Kubwa',
 'Shomolu',
 'Ogudu',
 'Isolo',
 'Wuye',
 'Ikorodu',
 'Apo',
 'Wuse 2',
 'Durumi',
 'Mabushi',
 'Ikotun',
 'Surulere',
 'Maryland',
 'Ipaja',
 'Karsana',
 'Wuse',
 'Kurudu',
 'Karmo',
 'Gbagada',
 'Maitama District',
 'Yaba',
 'Kukwaba',
 'Mbora (Nbora)',
 'Jabi',
 'Alimosho',
 'Kosofe',
 'Karshi',
 'Gudu',
 'Ayobo',
 'Ilupeju',
 'Ketu',
 'Kado',
 'Kyami',
 'Garki',
 'Karu',
 'Kafe',
 'Ojo',
 'Dakibiyu',
 'Amuwo Odofin',
 'Ijede',
 'Bwari',
 'Kagini',
 'Diplomatic Zones',
 'Oshodi',
 'Epe',
 'Mushin',
 'Oke-Odo',
 'Kabusa',
 'Dei-Dei',
 'Gwagwalada',
 'Egbe',
 'Duboyi',
 'Central Bus